In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework amuse-huayno"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

## Generic and Nbody unit systems

In this tutorial the generic and nbody systems are explained in more detail

In [ ]:
%matplotlib inline
from matplotlib import pyplot
import numpy

In the tutorial on units and quantities you have familiarized yourself with the use of units and the construction of quantities which are a combination of numerical values and an associated unit. The use of units in AMUSE serves two purposes: 1) it gives physical meaning to the results of simulation codes and the outcome of calculations and 2) they serves mechanism to check the consistency of the calculations. The physical meaning allows us two codes in AMUSE to understand each others output (e.g. conversion from m to parsec), the consistency check makes sure that the input are meaningful (ie no masses are used as positions).

In many community codes and for a lot of algorithms it makes sense to use units for which the base units are not (fully) specified. This is often the case when e.g. the equations solved are scale free or for initial conditions where similar models can be scaled to different sizes. Although the quantities involved in this case do not have a specific unit base, they still have a dimension (mass, length etc.). So in this case, the variables of a code, e.g. the position, do not have a particular scale, they still have the dimension length. In AMUSE we can use *generic units* in this case. In other words, you can specify if a value has a *mass*, *length* or *time* dimension, or any combination thereof, such as *length* per *time*. 

We will make this clearer with a few examples.

First imports:

In [ ]:
from amuse.units import units, constants, nbody_system, generic_unit_system

AMUSE includes two generic unit systems, the **generic_unit_system** is the most general, the **nbody_system** is a special case and defines the gravitational constant to have the numerical value `G=1` (as we will see G in this case still has dimensions!). For gravity calculations the **nbody_system** module is recommended as this follows the general practice in most n-body codes. Note that in the literature N-body units often have the additional property that he system being studied is scaled such that total mass `M=1` and energy `E=-0.25` (there are also called Hénon units).

The generic units are defined in the **generic_system** and **nbody_system** modules.

In [ ]:
print(10.0 | nbody_system.length)
print(10.0 | generic_unit_system.length)

Quantities with generic units work exactly the same as quantities with normal (**S.I.**) units.

In [ ]:
cluster_mass = 1.0 | generic_unit_system.length
mean_speed = 0.1 | generic_unit_system.length / generic_unit_system.time
print(mean_speed * cluster_mass)


Generic quantities are very useful and can be applied almost everywhere in AMUSE. 

To convert quantities to and from a specific system of units you'll need a converter. For nbody units you can create a converter to S.I. units like this:

In [ ]:
converter = nbody_system.nbody_to_si(1 | units.MSun, 1 | units.kms)

An ``nbody_system`` converter is specified by passing to it two independent quantities, which are to be scaled to 1. Together with the requirement that `G=1` this fixes the unit system. These quantities can be simple (like 1 solar mass) or combined (like  10 km/s). With the scaling the resulting object converts to and from the nbody units:

In [ ]:
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 nbody masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 nbody time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in nbody:", converter.to_nbody(10.0 | units.km / units.s))

Lets check the scaling explicitly: 

In [ ]:
print(f"For {converter}:")
print(f" G = {converter.to_nbody(constants.G)}")
print(f" 1 MSun = {converter.to_nbody(1 | units.MSun)}")
print(f" 1 km/s = {converter.to_nbody(1| units.kms)}")

As expected, the nbody units of these quantities all have the numerical value `1` (however they are not dimensionless! scale-free does not mean dimensionless!!)

It is important to understand what the converter does: it provides a particular scaling from a generic unit system to (most often) S.I. . In AMUSE we use it often in two roles: passed as an argument to a code class constructor it fixes the scaling of the input and output quantities of the code (be it variables or parameters). Passed as an argument to an initial condition generator it fixes the scaling of that initial condition generator. So this is very useful - but it is not strictly required! (we will give examples further down)   

For the generic unit converter, you can specify up to 7 quantities (as there are 7 base properties). Any combination of quantities is possible as long as it results in a orthogonal set of converters. 

In [ ]:
converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(1 | units.MSun, 1 | units.AU, constants.G)
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 generic masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 generic time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in generic:", converter.to_nbody(10.0 | units.km / units.s))

In [ ]:
converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(1 | units.MSun, 1 | units.AU, 1 | units.yr)
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 generic masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 generic time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in generic:", converter.to_nbody(10.0 | units.km / units.s))

Specifying a length twice or specifying a speed and a length and a time will result in an error. 

In [ ]:
generic_unit_converter.ConvertBetweenGenericAndSiUnits(
    1 | units.MSun, 
    1 | units.AU, 
    1 | units.m,
)

As an example, the following defines a converter for Planck units:

In [ ]:
natural_units_convert = generic_unit_converter.ConvertBetweenGenericAndSiUnits(
    constants.c,
    constants.G,
    constants.hbar,
    1/(4*numpy.pi*constants.eps0),
    constants.kB,
)

M = 1 | generic_unit_system.mass
T = 1 | generic_unit_system.time
L = 1 | generic_unit_system.length
Q = 1 | generic_unit_system.charge
THETA = 1 | generic_unit_system.temperature

print(natural_units_convert.to_si(M).in_(units.kg))
print(natural_units_convert.to_si(T).in_(units.s))
print(natural_units_convert.to_si(L).in_(units.m))
print(natural_units_convert.to_si(Q).in_(units.C))
print(natural_units_convert.to_si(THETA).in_(units.K))

Of course unit commensurability is still enforced:

In [ ]:
print((10.0 | nbody_system.length) + (10.0 | generic_unit_system.time))

In [ ]:
from amuse.ic.plummer import new_plummer_model
from amuse.community.huayno.interface import Huayno

### Assignment 1

Examine and try out the following:

In [ ]:
p=new_plummer_model(100)
h=Huayno()
print(h.parameters)
h.particles.add_particles(p)
h.evolve_model(1|nbody_system.time)
print(h.particles)
h.stop()

In [ ]:
convert = nbody_system.nbody_to_si(100. | units.MSun, 1. | units.parsec)
p=new_plummer_model(100, convert_nbody=convert)
h=Huayno(convert)
print(f"smoothing parameter: {h.parameters.epsilon_squared}")
h.particles.add_particles(p)
h.evolve_model(1| units.Myr)
print(h.particles)
h.stop()

In [ ]:
convert = nbody_system.nbody_to_si(100. | units.MSun, 1. | units.parsec)
p=new_plummer_model(100)
h=Huayno(convert)
print(f"smoothing parameter: {h.parameters.epsilon_squared}")
h.particles.add_particles(p)
h.evolve_model(1| nbody_system.time)
print(h.particles)
print(f" reached {h.model_time.in_(units.Myr)}")
h.stop()

### Assignment 2

Look up and listen to the Huayno preserved on the Voyager 2 Golden record.